In [1]:
from ipyleaflet import *
import requests
import urllib.parse as parse

addresses_filename = 'addresses.txt'
addresses = []

# Parse a line of full address to extract useful info.
# This helps improve geocoding performance.
# Returns dict with keys "full_address", "street_address", "city"
def parse_address(full_address):
    # Format: 150 Eglinton Avenue E, Suite 404 Toronto ON M4P 1E8
    try:
        split = line.rstrip().split(', ')
        street_address = split[0]
        city = split[-1].split()[-4]
        address = {'full_address': full_address,
                   'street_address': street_address, 
                   'city': city}
        return address
    except:
        print('Error parsing "{}".\n'.format(line))

# Look up the coordinates of an address.
# This is done by a selected geocoding API service.
# Returns tuple (latitude, longitude)
# Other APIs to consider: Nominatim
def lookup(address, api = 'positionstack'):
    if api == 'positionstack': # positionstack.com
        geocoding_base_url = 'http://api.positionstack.com/v1/forward?'
        geocoding_api_key = '109dccf957524b1ae1aa551a1ecc0138'
        params = {'access_key': geocoding_api_key, 
                  'query': address['street_address'], 
                  'region': address['city']}
        geocoding_url = geocoding_base_url + parse.urlencode(params)
        # print("Quering {}\n".format(address['street_address']))
        try:
            response = requests.get(geocoding_url)
            location = response.json()['data'][0] # select the first result
            return (location['latitude'], location['longitude'])
        except:
            print('Error finding "{}".\n'.format(address['street_address']))

# Draw address on map.
def draw_address(address, m):
    coordinate = (address['latitude'], address['longitude'])
    marker = Marker(location = coordinate, 
                    draggable = False, 
                    title = address['full_address'])
    m.add_layer(marker)


In [2]:
# Read file and populate addresses list.

with open(addresses_filename) as file:
    for line in file:
        address = parse_address(line)
        addresses.append(address)

for address in addresses:
    location = lookup(address)
    address['latitude'] = location[0]
    address['longitude'] = location[1]
    # print("Found coordinate {}, {}\n".format(location[0], location[1]))


In [3]:
# Draw addresses on map.

# TODO: find a suitable center and zoom. Check ipyleaflet.leaflet.Map.fit_bounds()
center = (addresses[0]['latitude'], addresses[0]['longitude'])
zoom = 10
m = Map(basemap = basemaps.Esri.WorldStreetMap, center = center, zoom = zoom)

for address in addresses:
    draw_address(address, m)
    
display(m)


Map(center=[43.707906, -79.394007], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…